In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

I want to calculate accuracies

FOr the three task model and the rendezvous

In [2]:
import os
import json

def load_ground_truth_components(labelme_path, mode="triplet"):
    with open(labelme_path, 'r') as f:
        data = json.load(f)

    gt_items = set()
    for shape in data.get("shapes", []):
        instrument = shape.get("label")
        verb = shape.get("verb")
        target = shape.get("target")

        if mode == "instrument":
            if instrument: gt_items.add(instrument)
        elif mode == "verb":
            if verb: gt_items.add(verb)
        elif mode == "target":
            if target: gt_items.add(target)
        elif mode == "triplet":
            if instrument and verb and target:
                gt_items.add(f"{instrument},{verb},{target}")
    
    return gt_items


In [3]:
import os
import json

def load_ground_truth_components(labelme_path, mode="triplet"):
    with open(labelme_path, 'r') as f:
        data = json.load(f)

    gt_items = set()
    for shape in data.get("shapes", []):
        instrument = shape.get("label")
        verb = shape.get("verb")
        target = shape.get("target")

        if mode == "instrument":
            if instrument: gt_items.add(instrument)
        elif mode == "verb":
            if verb: gt_items.add(verb)
        elif mode == "target":
            if target: gt_items.add(target)
        elif mode == "triplet":
            if instrument and verb and target:
                gt_items.add(f"{instrument},{verb},{target}")
    
    return gt_items


def evaluate_triplet_predictions(prediction_json_path, labelme_dir, mode="triplet"):
    """
    Evaluates prediction performance across images.
    
    Args:
        prediction_json_path: Path to prediction file.
        labelme_dir: Directory with LabelMe-style ground truth JSONs.
        mode: 'instrument', 'verb', 'target', or 'triplet'.

    Returns:
        Dictionary with accuracy, precision, recall, f1, tp, fp, fn, total_images, correct_images
    """
    with open(prediction_json_path, 'r') as f:
        predictions = json.load(f)

    total_images = 0
    correct_images = 0
    total_tp = 0
    total_fp = 0
    total_fn = 0

    for image_name, pred_data in predictions.items():
        labelme_path = os.path.join(labelme_dir, f"t50_{image_name}.json")
        if not os.path.exists(labelme_path):
            print(f"Skipping {image_name} (no labelme file found)")
            continue

        gt_items = load_ground_truth_components(labelme_path, mode=mode)

        # Extract prediction items
        pred_triplets = pred_data["triplet_name"]
        if mode == "instrument":
            pred_items = {t.split(",")[0] for t in pred_triplets if "," in t}
        elif mode == "verb":
            pred_items = {t.split(",")[1] for t in pred_triplets if "," in t}
        elif mode == "target":
            pred_items = {t.split(",")[2] for t in pred_triplets if "," in t}
        elif mode == "triplet":
            pred_items = set(pred_triplets)

        tp = len(gt_items & pred_items)
        fp = len(pred_items - gt_items)
        fn = len(gt_items - pred_items)

        total_tp += tp
        total_fp += fp
        total_fn += fn

        total_images += 1
        if tp > 0:
            correct_images += 1

    # Image-level accuracy
    accuracy = total_tp / (total_tp + total_fp + total_fn)  if total_images > 0 else 0.0

    # Precision, recall, f1
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

    print(f"\n{mode.title()} Evaluation:")
    print(f"  Accuracy:  {accuracy:.4f} ")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"  TP: {total_tp}, FP: {total_fp}, FN: {total_fn}")
    
    return



Why are there so many false negatives in my model?? Makes no sense really. 

In [4]:
ground_truth_dir =  '../../datasets/my_triplet_seg_datasets/triplet_segmentation_dataset_v2_second_stage/test/ann_dir/'
rendezvous_json_path = '../resnet_model/work_dirs/rendezvous_cholectriplet_seg/updated_rendezvous_results.json'
my_direct_model_json_path = '../results/mask2former_direct_triplet_prediction/results_triplet_softmax_scores.json'
my_model_json_path = '../resnet_model/work_dirs/threetask_resnet_fpn_parallel_decoders/results_triplet_softmax_scores.json'

In [7]:
evaluate_triplet_predictions(rendezvous_json_path, ground_truth_dir, mode="instrument")
evaluate_triplet_predictions(rendezvous_json_path, ground_truth_dir, mode="verb")
evaluate_triplet_predictions(rendezvous_json_path, ground_truth_dir, mode="target")
evaluate_triplet_predictions(rendezvous_json_path, ground_truth_dir, mode="triplet")


Instrument Evaluation:
  Accuracy:  0.5669 
  Precision: 0.8829
  Recall:    0.6130
  F1 Score:  0.7236
  TP: 3512, FP: 466, FN: 2217

Verb Evaluation:
  Accuracy:  0.5078 
  Precision: 0.8089
  Recall:    0.5770
  F1 Score:  0.6736
  TP: 3281, FP: 775, FN: 2405

Target Evaluation:
  Accuracy:  0.4126 
  Precision: 0.6563
  Recall:    0.5263
  F1 Score:  0.5842
  TP: 2641, FP: 1383, FN: 2377

Triplet Evaluation:
  Accuracy:  0.3323 
  Precision: 0.5688
  Recall:    0.4442
  F1 Score:  0.4989
  TP: 2545, FP: 1929, FN: 3184


In [5]:
evaluate_triplet_predictions(my_model_json_path, ground_truth_dir, mode="instrument")
evaluate_triplet_predictions(my_model_json_path, ground_truth_dir, mode="verb")
evaluate_triplet_predictions(my_model_json_path, ground_truth_dir, mode="target")
evaluate_triplet_predictions(my_model_json_path, ground_truth_dir, mode="triplet")


Instrument Evaluation:
  Accuracy:  0.9107 
  Precision: 0.9637
  Recall:    0.9430
  F1 Score:  0.9533
  TP: 5363, FP: 202, FN: 324

Verb Evaluation:
  Accuracy:  0.7380 
  Precision: 0.8551
  Recall:    0.8436
  F1 Score:  0.8493
  TP: 4761, FP: 807, FN: 883

Target Evaluation:
  Accuracy:  0.4704 
  Precision: 0.6413
  Recall:    0.6383
  F1 Score:  0.6398
  TP: 3177, FP: 1777, FN: 1800

Triplet Evaluation:
  Accuracy:  0.4103 
  Precision: 0.5857
  Recall:    0.5780
  F1 Score:  0.5818
  TP: 3287, FP: 2325, FN: 2400


In [6]:
evaluate_triplet_predictions(my_direct_model_json_path, ground_truth_dir, mode="instrument")
evaluate_triplet_predictions(my_direct_model_json_path, ground_truth_dir, mode="verb")
evaluate_triplet_predictions(my_direct_model_json_path, ground_truth_dir, mode="target")
evaluate_triplet_predictions(my_direct_model_json_path, ground_truth_dir, mode="triplet")


Instrument Evaluation:
  Accuracy:  0.8800 
  Precision: 0.9548
  Recall:    0.9183
  F1 Score:  0.9362
  TP: 5066, FP: 240, FN: 451

Verb Evaluation:
  Accuracy:  0.7212 
  Precision: 0.8350
  Recall:    0.8411
  F1 Score:  0.8380
  TP: 4604, FP: 910, FN: 870

Target Evaluation:
  Accuracy:  0.5173 
  Precision: 0.6626
  Recall:    0.7023
  F1 Score:  0.6819
  TP: 3380, FP: 1721, FN: 1433

Triplet Evaluation:
  Accuracy:  0.4605 
  Precision: 0.6083
  Recall:    0.6547
  F1 Score:  0.6306
  TP: 3612, FP: 2326, FN: 1905
